In [7]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import random
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score

In [8]:
torch.manual_seed(42)
np.random.seed(42)

In [3]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [9]:
torch.manual_seed(42)
np.random.seed(42)

### Util function

In [33]:
def load_dataset_timestamp( n_users, n_context, seq_len):
    act_list = []
    time_list = []
    user_list = []

    max_timestamp = -1.0
    min_timestamp = float('inf')

    with open('gowalla_user_activity.txt', 'r') as raw_file:
        for line in raw_file:
            t_item_list = []
            t_time_list = []
            user = int(line.split(':')[0])
            entries = line.split()[1:]
            for a_entry in entries:
                item, time_stamp = a_entry.split(':')
                t_item_list.append(int(item.strip()))
                t_time_list.append(int(time_stamp.strip()))

                if min_timestamp > int(time_stamp.strip()):
                    min_timestamp = int(time_stamp.strip())
                if max_timestamp < int(time_stamp.strip()):
                    max_timestamp = int(time_stamp.strip())

            act_list.append(t_item_list[0: seq_len])
            time_list.append(t_time_list[0: seq_len])
            user_list.append(user)

    new_time_list = []
    num_bins = 0

    times_bins = np.linspace(min_timestamp, max_timestamp + 1, num=num_bins, dtype=np.int32)
    for a_time_list in time_list:
        temp_time_list = (np.digitize(np.asarray(a_time_list), times_bins) - 1).tolist()
        new_time_list.append(temp_time_list)

    all_examples = []
    for i in range(0, len(act_list)):

        train_act_seq = act_list[i][:-2]

        train_time_seq = new_time_list[i][:-2]

        train_act_label = act_list[i][-2]
        train_time_label = new_time_list[i][-2]

        test_act_seq = act_list[i][1:-1]
        test_time_seq = new_time_list[i][1:-1]

        test_act_label = act_list[i][-1]
        test_time_label = new_time_list[i][-1]

        entry = {
            'train_act_seq': train_act_seq,
            'train_time_seq': train_time_seq,
            'train_act_label': train_act_label,
            'train_time_label': train_time_label,
            'test_act_seq': test_act_seq,
            'test_time_seq': test_time_seq,
            'test_act_label': test_act_label,
            'test_time_label': test_time_label,
            'seq_len': len(train_act_seq),
            'user': user_list[i]
        }

        all_examples.append(entry)

    return all_examples

In [34]:
data_examples = load_dataset_timestamp(20001, 128, 30)

In [35]:
def filter(x):
  return [user for user in users if user not in x]

### Metrics

In [36]:
def print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100):
    print(f'hits@1: {hits1:.6f}, hits@5: {hits5:.6f}, hits@10: {hits10:.6f}, hits@20: {hits20:.6f}')
    print(f'hits@50: {hits50:.6f}, hits@100: {hits100:.6f}')
    print(f'map@1: {map1:.6f}, map@5: {map5:.6f}, map@10: {map10:.6f}, map@20: {map20:.6f}')
    print(f'map@50: {map50:.6f}, map@100: {map100:.6f}')
    print(f'ndcg@1: {ndcg1:.6f}, ndcg@5: {ndcg5:.6f}, ndcg@10: {ndcg10:.6f}, ndcg@20: {ndcg20:.6f}')
    print(f'ndcg@50: {ndcg50:.6f}, ndcg@100: {ndcg100:.6f}')

In [37]:
def apk(actual, predicted, k=10):

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(y_prob, y, k=10):

    predicted = [np.argsort(p_)[-k:][::-1] for p_ in y_prob]
    actual = [[y_] for y_ in y]
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def hits_k(y_prob, y, k=10):
    acc = []
    for p_, y_ in zip(y_prob, y):
        top_k = p_.argsort()[-k:][::-1]
        acc += [1. if y_ in top_k else 0.]
from sklearn.metrics import ndcg_score


In [38]:
def get_metrics_(probs, labels_batch, test_one_hot):
    hits1 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=1, labels = classes)
    hits5 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=5, labels = classes)
    hits10 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=10, labels = classes)
    hits20 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=20, labels = classes)
    hits50= top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=50, labels = classes)
    hits100 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=100, labels = classes)

    map1 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=1)
    map5 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=5)
    map10 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=10)
    map20 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=20)
    map50 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=50)
    map100 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=100)

    ndcg1 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=1)
    ndcg5 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=5)
    ndcg10 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=10)
    ndcg20 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=20)
    ndcg50 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=50)
    ndcg100 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=100)
    return hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100
    return sum(acc) / len(acc)

### Pairs construction

In [39]:
friends = pd.read_csv('gowalla_edges.csv')
relationships = friends.groupby('1st friend').agg({'2nd friend': lambda x: list(x)}).rename(columns={'2nd friend':'friends'})
users = [x for x in range(0, 20001)]
relationships['not friends'] = relationships['friends'].map(lambda x: filter(x))
relationships = relationships.reset_index()
relationships.apply(lambda x: x['not friends'].remove(x['1st friend']), axis = 1)

0        None
1        None
2        None
3        None
4        None
         ... 
19721    None
19722    None
19723    None
19724    None
19725    None
Length: 19726, dtype: object

In [40]:
pairs = []
num_users = len(users)
for i in range(relationships.shape[0]):
  user = relationships.loc[i, :]
  if len(user['friends']) >= 1:
    random_friends = random.sample(user['friends'], 1)
    random_not_friends = random.sample(user['not friends'], 1)
  else:
    random_friends = user['friends']
    random_not_friends = random.sample(user['not friends'], 1)
  for friend in random_friends:
    pairs.append((user['1st friend'], friend, 1))
  for noname in random_not_friends:
    pairs.append((user['1st friend'], noname, 0))

for i in range(0, num_users):
  if i not in relationships['1st friend'].tolist():
    not_friends = [x for x in range(0, num_users)]
    not_friends.remove(i)
    random_not_friends = random.sample(not_friends, 1)
  for noname in random_not_friends:
    pairs.append((i, noname, 0))

In [41]:
class PairDataset():
  def __init__(self, pairs, data, max_len):
    self.pairs = pairs
    self.data = data
    self.max_len = max_len

  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, idx):
    first_friend = self.pairs[idx][0]
    second_friend = self.pairs[idx][1]
    link = self.pairs[idx][2]
    node_1 = self.data[first_friend]
    node_2 = self.data[second_friend]
    seq_len_1 = node_1['seq_len']
    seq_len_2 = node_2['seq_len']

    tr_act_seq_1 = np.zeros((self.max_len,)).astype('int32')
    tr_act_seq_1[:seq_len_1] = np.array(node_1['train_act_seq'])
    tr_act_seq_1 = np.transpose(tr_act_seq_1)

    tr_time_seq_1 = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq_1[:seq_len_1] = node_1['train_time_seq']
    tr_time_seq_1 = np.transpose(tr_time_seq_1)

    t_act_seq_1 = np.zeros((self.max_len, )).astype('int32')
    t_act_seq_1[:seq_len_1] = node_1['test_act_seq']
    t_act_seq_1 = np.transpose(t_act_seq_1)

    t_time_seq_1 = np.zeros((self.max_len, )).astype('int32')
    t_time_seq_1[:seq_len_1] = node_1['test_time_seq']
    t_time_seq_1 = np.transpose(t_time_seq_1)

    tr_act_seq_2 = np.zeros((self.max_len, )).astype('int32')
    tr_act_seq_2[:seq_len_2] = node_2['train_act_seq']
    tr_act_seq_2 = np.transpose(tr_act_seq_2)

    tr_time_seq_2 = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq_2[:seq_len_2] = node_2['train_time_seq']
    tr_time_seq_2 = np.transpose(tr_time_seq_2)

    t_act_seq_2 = np.zeros((self.max_len,)).astype('int32')
    t_act_seq_2[:seq_len_2] = node_2['test_act_seq']
    t_act_seq_2 = np.transpose(t_act_seq_2)

    t_time_seq_2 = np.zeros((self.max_len,)).astype('int32')
    t_time_seq_2[:seq_len_2] = node_2['test_time_seq']
    t_time_seq_2 = np.transpose(t_time_seq_2)

    return node_1['user'], tr_act_seq_1, \
    tr_time_seq_1, node_1['train_act_label'], \
    node_1['train_time_label'], t_act_seq_1, \
    t_time_seq_1, node_1['test_act_label'], \
    node_1['test_time_label'], node_1['seq_len'], \
    node_2['user'], tr_act_seq_2, \
    tr_time_seq_2, node_2['train_act_label'], \
    node_2['train_time_label'], t_act_seq_2, \
    t_time_seq_2, node_2['test_act_label'], \
    node_2['test_time_label'], node_2['seq_len'], link

In [42]:
pair_dataset = PairDataset(pairs, data_examples, 30)
user_1, train_input_1, train_time_1, train_label_1, train_time_label_1, test_input_1, test_time_1, test_label_1, test_time_label_1, seq_len_1, \
user_2, train_input_2, train_time_2, train_label_2, train_time_label_2, test_input_2, test_time_2, test_label_2, test_time_label_2, seq_len_2, \
link = pair_dataset[0]

In [43]:
class UserDataset():
    def __init__(self, user_representations):
        self.user_representations = user_representations

    def __len__(self):
        return len(self.user_representations)

    def __getitem__(self, idx):
        user_id = idx
        if idx not in self.user_representations.keys():
          return user_id, self.user_representations[1][0], 0
        return user_id, self.user_representations[idx][0], self.user_representations[idx][1]

In [51]:
num_classes = 186
classes = np.arange(0, num_classes)

### Target prediction + Link prediction model

In [52]:
class TargetWithLinkPredictionModel(nn.Module):

  def __init__(self):
    super(TargetWithLinkPredictionModel, self).__init__()

    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.Tanh()
    self.norm = nn.BatchNorm1d(128)
    self.head_1 = nn.Sequential(nn.Linear(256, 2))
    self.head_2 = nn.Linear(128, num_classes)
    self.sigmoid= nn.Sigmoid()

  def forward(self, x_1, x_2, seq_len_1, seq_len_2):

    x_1, h_1 = self.rnn(x_1)
    hx_1 = torch.zeros(x_1.shape[0], x_1.shape[2])
    for i in range(hx_1.shape[0]):
      hx_1[i] = x_1[i][seq_len_1[i] - 1]
    hx_1 = self.norm(hx_1)

    x_2, h_2 = self.rnn(x_2)
    hx_2 = torch.zeros(x_2.shape[0], x_2.shape[2])
    for i in range(hx_2.shape[0]):
      hx_2[i] = x_2[i][seq_len_2[i] - 1]
    hx_2 = self.norm(hx_2)

    x = torch.cat((hx_1, hx_2), dim = 1)
    link_pred = self.head_1(x)
    next_pred = self.head_2(hx_1)
    return link_pred, next_pred, hx_1, hx_2

### Dataset construction

In [53]:
pair_dataset = PairDataset(pairs, data_examples, 30)

In [54]:
n = len(pair_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

pair_train_dataset = Subset(pair_dataset, train_indices)
pair_test_dataset = Subset(pair_dataset, test_indices)

pair_train_dataloader = DataLoader(pair_train_dataset, batch_size=64, shuffle=True)
pair_test_dataloader = DataLoader(pair_test_dataset, batch_size=64, shuffle=True)

In [55]:
item_emb  = nn.init.xavier_uniform_(torch.empty(num_classes, 128))

### Target prediction + Link prediction training

In [56]:
link_loss_fn = nn.CrossEntropyLoss()
next_loss_fn = nn.CrossEntropyLoss()
model  = TargetWithLinkPredictionModel()

train_user_representations = {}
test_user_representations = {}

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)
epochs = 10


for i in range(epochs):
  metrics_val = []
  hits_1_scores = []
  losses = []

  for user_1, train_input_1, train_time_1, train_label_1, train_time_label_1, test_input_1, test_time_1, test_label_1, test_time_label_1, seq_len_1, \
      user_2, train_input_2, train_time_2, train_label_2, train_time_label_2, test_input_2, test_time_2, test_label_2, test_time_label_2, seq_len_2, \
      link in pair_train_dataloader:

      optimizer.zero_grad()


      comb_input_1 = np.concatenate([np.expand_dims(train_input_1, axis=-1),
                                                np.expand_dims(train_time_1, axis=-1)], axis=2)
      comb_input_2 = np.concatenate([np.expand_dims(train_input_2, axis=-1),
                                                np.expand_dims(train_time_2, axis=-1)], axis=2)
      model_input_1 = comb_input_1
      model_input_2 = comb_input_2

      model_output = link
      rnn_input_emb_1 = item_emb[model_input_1[:, :, 0]]
      rnn_input_emb_2 = item_emb[model_input_2[:, :, 0]]

      seq_len_1 = torch.Tensor(seq_len_1).to(torch.int32)
      seq_len_2 = torch.Tensor(seq_len_2).to(torch.int32)


      link_prob, next_prob, hx_1, hx_2 = model(rnn_input_emb_1, rnn_input_emb_2, seq_len_1, seq_len_2)

      pred = torch.argmax(link_prob, axis = 1)
      one_hot = torch.zeros(len(link_prob), 2)
      one_hot[torch.arange(len(one_hot)), model_output] = 1
      model_output = torch.Tensor(model_output).view(1, -1).permute(1, 0)

      link_loss = link_loss_fn(link_prob, one_hot)
      next_one_hot = torch.zeros(len(next_prob), 186)
      next_one_hot[torch.arange(len(next_one_hot)), train_label_1] = 1
      next_loss = next_loss_fn(next_prob, next_one_hot)

      loss = link_loss + 1.5 * next_loss

      losses.append(loss)

      loss.backward()
      optimizer.step()

      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100,\
      ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = get_metrics_(next_prob, train_label_1, next_one_hot)
      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])

  mean = torch.Tensor(metrics_val).mean(axis=0)
  mean_loss = torch.Tensor(losses).mean(axis=0).item()
  print(f'Epoch: {i}, Loss:{mean_loss}')
  print('Training metrics')
  hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, \
  ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = mean
  print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)

Epoch: 0, Loss:2.056633472442627
Training metrics
hits@1: 0.811559, hits@5: 0.924878, hits@10: 0.941931, hits@20: 0.956254
hits@50: 0.973265, hits@100: 0.985341
map@1: 0.811559, map@5: 0.859818, map@10: 0.862153, map@20: 0.863147
map@50: 0.863710, map@100: 0.863886
ndcg@1: 0.811559, ndcg@5: 0.876356, ndcg@10: 0.881929, ndcg@20: 0.885551
ndcg@50: 0.888962, ndcg@100: 0.890930
Epoch: 1, Loss:1.566477656364441
Training metrics
hits@1: 0.836719, hits@5: 0.961084, hits@10: 0.978537, hits@20: 0.989814
hits@50: 0.996745, hits@100: 0.999307
map@1: 0.836719, map@5: 0.889194, map@10: 0.891583, map@20: 0.892390
map@50: 0.892625, map@100: 0.892662
ndcg@1: 0.836719, ndcg@5: 0.907439, ndcg@10: 0.913143, ndcg@20: 0.916024
ndcg@50: 0.917425, ndcg@100: 0.917842
Epoch: 2, Loss:1.4760764837265015
Training metrics
hits@1: 0.841003, hits@5: 0.966684, hits@10: 0.984606, hits@20: 0.993679
hits@50: 0.999013, hits@100: 0.999832
map@1: 0.841003, map@5: 0.894248, map@10: 0.896726, map@20: 0.897385
map@50: 0.89756

### Link prediction model

In [57]:
class LinkPredictionModel(nn.Module):

  def __init__(self):
    super(LinkPredictionModel, self).__init__()

    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.Tanh()
    self.norm = nn.BatchNorm1d(128)
    self.head_1 = nn.Sequential(nn.Linear(256, 2))
    self.head_2 = nn.Linear(128, num_classes)
    self.sigmoid= nn.Sigmoid()

  def forward(self, x_1, x_2, seq_len_1, seq_len_2):

    x_1, h_1 = self.rnn(x_1)
    hx_1 = torch.zeros(x_1.shape[0], x_1.shape[2])
    for i in range(hx_1.shape[0]):
      hx_1[i] = x_1[i][seq_len_1[i] - 1]
    hx_1 = self.norm(hx_1)

    x_2, h_2 = self.rnn(x_2)
    hx_2 = torch.zeros(x_2.shape[0], x_2.shape[2])
    for i in range(hx_2.shape[0]):
      hx_2[i] = x_2[i][seq_len_2[i] - 1]
    hx_2 = self.norm(hx_2)

    x = torch.cat((hx_1, hx_2), dim = 1)
    link_pred = self.head_1(x)
    return link_pred, hx_1, hx_2

Teaining link prediction model

In [58]:
link_loss_fn = nn.CrossEntropyLoss()
next_loss_fn = nn.CrossEntropyLoss()
model  = LinkPredictionModel()

train_user_representations = {}
test_user_representations = {}

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)
epochs = 10


for i in range(epochs):
  metrics_val = []
  accuracy_scores = []
  losses = []

  for user_1, train_input_1, train_time_1, train_label_1, train_time_label_1, test_input_1, test_time_1, test_label_1, test_time_label_1, seq_len_1, \
      user_2, train_input_2, train_time_2, train_label_2, train_time_label_2, test_input_2, test_time_2, test_label_2, test_time_label_2, seq_len_2, \
      link in pair_train_dataloader:

      optimizer.zero_grad()


      comb_input_1 = np.concatenate([np.expand_dims(train_input_1, axis=-1),
                                                np.expand_dims(train_time_1, axis=-1)], axis=2)
      comb_input_2 = np.concatenate([np.expand_dims(train_input_2, axis=-1),
                                                np.expand_dims(train_time_2, axis=-1)], axis=2)
      model_input_1 = comb_input_1
      model_input_2 = comb_input_2

      model_output = link
      rnn_input_emb_1 = item_emb[model_input_1[:, :, 0]]
      rnn_input_emb_2 = item_emb[model_input_2[:, :, 0]]

      seq_len_1 = torch.Tensor(seq_len_1).to(torch.int32)
      seq_len_2 = torch.Tensor(seq_len_2).to(torch.int32)


      link_prob, hx_1, hx_2 = model(rnn_input_emb_1, rnn_input_emb_2, seq_len_1, seq_len_2)

      pred = torch.argmax(link_prob, axis = 1)
      one_hot = torch.zeros(len(link_prob), 2)
      one_hot[torch.arange(len(one_hot)), model_output] = 1
      model_output = torch.Tensor(model_output).view(1, -1).permute(1, 0)

      link_loss = link_loss_fn(link_prob, one_hot)

      loss = link_loss
      accuracy_scores.append(accuracy_score(model_output, pred))

      losses.append(loss)

      loss.backward()
      optimizer.step()

  mean = torch.Tensor(metrics_val).mean(axis=0)
  mean_loss = torch.Tensor(losses).mean(axis=0).item()
  mean_accuracy = torch.Tensor(accuracy_scores).mean(axis=0).item()
  print(f'Epoch: {i}, Loss:{mean_loss} Train accuracy: {mean_accuracy}')

Epoch: 0, Loss:0.49598291516304016 Train accuracy: 0.7258568406105042
Epoch: 1, Loss:0.47170543670654297 Train accuracy: 0.7419102787971497
Epoch: 2, Loss:0.4813894033432007 Train accuracy: 0.7369371652603149
Epoch: 3, Loss:0.4596621096134186 Train accuracy: 0.74403977394104
Epoch: 4, Loss:0.44971615076065063 Train accuracy: 0.7495001554489136
Epoch: 5, Loss:0.44246309995651245 Train accuracy: 0.7533811926841736
Epoch: 6, Loss:0.43718650937080383 Train accuracy: 0.7574806809425354
Epoch: 7, Loss:0.4316729009151459 Train accuracy: 0.7580267190933228
Epoch: 8, Loss:0.42781516909599304 Train accuracy: 0.7629326581954956
Epoch: 9, Loss:0.4230033755302429 Train accuracy: 0.766427218914032


Obtaining representations of users from training part

In [59]:
train_user_representations = {}
test_user_representations = {}

for user_1, train_input_1, train_time_1, train_label_1, train_time_label_1, test_input_1, test_time_1, test_label_1, test_time_label_1, seq_len_1, \
      user_2, train_input_2, train_time_2, train_label_2, train_time_label_2, test_input_2, test_time_2, test_label_2, test_time_label_2, seq_len_2, \
      link in pair_train_dataloader:

  comb_input_1 = np.concatenate([np.expand_dims(train_input_1, axis=-1),
                                                  np.expand_dims(train_time_1, axis=-1)], axis=2)
  comb_input_2 = np.concatenate([np.expand_dims(train_input_2, axis=-1),
                                            np.expand_dims(train_time_2, axis=-1)], axis=2)

  test_comb_input_1 = np.concatenate([np.expand_dims(test_input_1, axis=-1),
                                                  np.expand_dims(test_time_1, axis=-1)], axis=2)
  test_comb_input_2 = np.concatenate([np.expand_dims(test_input_2, axis=-1),
                                            np.expand_dims(test_time_2, axis=-1)], axis=2)

  model_input_1 = comb_input_1
  model_input_2 = comb_input_2

  test_model_input_1 = test_comb_input_1
  test_model_input_2 = test_comb_input_2

  model_output = link
  rnn_input_emb_1 = item_emb[model_input_1[:, :, 0]]
  rnn_input_emb_2 = item_emb[model_input_2[:, :, 0]]

  test_rnn_input_emb_1 = item_emb[test_model_input_1[:, :, 0]]
  test_rnn_input_emb_2 = item_emb[test_model_input_2[:, :, 0]]

  seq_len_1 = torch.Tensor(seq_len_1).to(torch.int32)

  seq_len_2 = torch.Tensor(seq_len_2).to(torch.int32)

  prob, x_1, x_2 = model(rnn_input_emb_1, rnn_input_emb_2, seq_len_1, seq_len_2)
  test_prob, t_x_1, t_x_2 = model(test_rnn_input_emb_1, test_rnn_input_emb_2, seq_len_1, seq_len_2)

  model_output = torch.Tensor(model_output).view(1, -1).permute(1, 0)

  index_1 = 0
  for user in user_1:
    user = user.item()
    if user not in train_user_representations.keys():
      train_user_representations[user] = (x_1[index_1].detach(), train_label_1[index_1])
    if user not in test_user_representations.keys():
      test_user_representations[user] = (t_x_1[index_1].detach(), test_label_1[index_1])
    index_1 += 1
  index_2 = 0
  for user in user_2:
    user = user.item()
    if user not in train_user_representations.keys():
      train_user_representations[user] = (x_2[index_2].detach(), train_label_2[index_2])
    if user not in test_user_representations.keys():
      test_user_representations[user] = (t_x_2[index_2].detach(), test_label_2[index_2])
    index_2 += 1

Obtaining representations of users from testing part

In [60]:
accuracy_scores = []
hits_scores = []

for user_1, train_input_1, train_time_1, train_label_1, train_time_label_1, test_input_1, test_time_1, test_label_1, test_time_label_1, seq_len_1, \
      user_2, train_input_2, train_time_2, train_label_2, train_time_label_2, test_input_2, test_time_2, test_label_2, test_time_label_2, seq_len_2, \
      link in pair_test_dataloader:

  comb_input_1 = np.concatenate([np.expand_dims(train_input_1, axis=-1),
                                                  np.expand_dims(train_time_1, axis=-1)], axis=2)
  comb_input_2 = np.concatenate([np.expand_dims(train_input_2, axis=-1),
                                            np.expand_dims(train_time_2, axis=-1)], axis=2)

  test_comb_input_1 = np.concatenate([np.expand_dims(test_input_1, axis=-1),
                                                  np.expand_dims(test_time_1, axis=-1)], axis=2)
  test_comb_input_2 = np.concatenate([np.expand_dims(test_input_2, axis=-1),
                                            np.expand_dims(test_time_2, axis=-1)], axis=2)

  model_input_1 = comb_input_1
  model_input_2 = comb_input_2

  test_model_input_1 = test_comb_input_1
  test_model_input_2 = test_comb_input_2

  model_output = link

  rnn_input_emb_1 = item_emb[model_input_1[:, :, 0]]
  rnn_input_emb_2 = item_emb[model_input_2[:, :, 0]]

  test_rnn_input_emb_1 = item_emb[test_model_input_1[:, :, 0]]
  test_rnn_input_emb_2 = item_emb[test_model_input_2[:, :, 0]]

  seq_len_1 = torch.Tensor(seq_len_1).to(torch.int32)

  seq_len_2 = torch.Tensor(seq_len_2).to(torch.int32)

  prob, x_1, x_2 = model(rnn_input_emb_1, rnn_input_emb_2, seq_len_1, seq_len_2)
  test_prob, t_x_1, t_x_2 = model(test_rnn_input_emb_1, test_rnn_input_emb_2, seq_len_1, seq_len_2)

  pred = torch.argmax(test_prob, axis = 1)
  model_output = torch.Tensor(model_output).view(1, -1).permute(1, 0)
  accuracy_scores.append(accuracy_score(model_output, pred))

  index_1 = 0
  for user in user_1:
    user = user.item()
    if user not in train_user_representations.keys():
      train_user_representations[user] = (x_1[index_1].detach(), train_label_1[index_1])
    if user not in test_user_representations.keys():
      test_user_representations[user] = (t_x_1[index_1].detach(), test_label_1[index_1])
    index_1 += 1

  index_2 = 0
  for user in user_2:
    user = user.item()
    if user not in train_user_representations.keys():
      train_user_representations[user] = (x_2[index_2].detach(), train_label_2[index_2])
    if user not in test_user_representations.keys():
      test_user_representations[user] = (t_x_2[index_2].detach(), test_label_2[index_2])
    index_2 += 1

mean = torch.Tensor(accuracy_scores).mean(axis=0).item()
print(f'Test accuracy: {mean}')

Test accuracy: 0.6769729852676392


### User representationsa dataset construction

In [61]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

### Target task training

In [62]:
class TargetModel(nn.Module):
  def __init__(self):
    super(TargetModel, self).__init__()
    self.fc1 = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.fc1(x)

    return x

In [64]:
def test(model):
  metrics_val = []
  model.eval()

  for users, vectors, labels in user_test_dataloader:

      test_probs = model(vectors)

      test_pred = torch.argmax(test_probs, axis = 1)

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), labels] = 1

      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100,\
      ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = get_metrics_(test_probs, labels, test_one_hot)
      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])

  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5,test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean

  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100,\
  test_ndcg1, test_ndcg5,test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100

In [65]:
loss_fn = nn.CrossEntropyLoss()
model  = TargetModel()

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)
losses = []

for i in range(30):
  for users, vectors, labels in user_train_dataloader:

      optimizer.zero_grad()

      probs = model(vectors)
      pred = torch.argmax(probs, axis = 1)
      one_hot = torch.zeros(len(probs), num_classes)
      one_hot[torch.arange(len(one_hot)), labels] = 1
      loss = loss_fn(probs, one_hot)
      loss.backward()
      optimizer.step()

hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, \
ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = test(model)
print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)

hits@1: 0.814852, hits@5: 0.900358, hits@10: 0.918681, hits@20: 0.937530
hits@50: 0.967773, hits@100: 0.989087
map@1: 0.814852, map@5: 0.851421, map@10: 0.853906, map@20: 0.855193
map@50: 0.856123, map@100: 0.856433
ndcg@1: 0.814852, ndcg@5: 0.863862, ndcg@10: 0.869826, ndcg@20: 0.874566
ndcg@50: 0.880507, ndcg@100: 0.883979
